网页浏览行为关联规则挖掘

数据集来源：http://archive.ics.uci.edu/ml/datasets/Anonymous+Microsoft+Web+Data

0 首先检查是否缺失必备的包，并补充安装

In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import warnings
warnings.filterwarnings('ignore')

查看数据概览

In [2]:
data=[]
with open('anonymous-msweb.data', 'r') as f_input:
    for line in f_input:
        data.append(list(line.strip().split(',')))
data

[['I', '4', '"www.microsoft.com"', '"created by getlog.pl"'],
 ['T', '1', '"VRoot"', '0', '0', '"VRoot"'],
 ['N', '0', '"0"'],
 ['N', '1', '"1"'],
 ['T', '2', '"Hide1"', '0', '0', '"Hide"'],
 ['N', '0', '"0"'],
 ['N', '1', '"1"'],
 ['A', '1287', '1', '"International AutoRoute"', '"/autoroute"'],
 ['A', '1288', '1', '"library"', '"/library"'],
 ['A', '1289', '1', '"Master Chef Product Information"', '"/masterchef"'],
 ['A', '1297', '1', '"Central America"', '"/centroam"'],
 ['A', '1215', '1', '"For Developers Only Info"', '"/developer"'],
 ['A', '1279', '1', '"Multimedia Golf"', '"/msgolf"'],
 ['A', '1239', '1', '"Microsoft Consulting"', '"/msconsult"'],
 ['A', '1282', '1', '"home"', '"/home"'],
 ['A', '1251', '1', '"Reference Support"', '"/referencesupport"'],
 ['A', '1121', '1', '"Microsoft Magazine"', '"/magazine"'],
 ['A', '1083', '1', '"MS Access Support"', '"/msaccesssupport"'],
 ['A', '1145', '1', '"Visual Fox Pro Support"', '"/vfoxprosupport"'],
 ['A', '1276', '1', '"Visual Test

整理数据形式

In [3]:
with open('anonymous-msweb.data', 'r') as f:
   data = f.readlines()
##选择A开头的网站数据，存入Web_data中
##选择C开头的用户数据，存入user_data中
##在V开头的数据后，添加V行的用户ID信息，存入visit_data，表示用户访问网站情况
a_lines = [line for line in data if line.startswith('A')]
c_lines = [line for line in data if line.startswith('C')]
a_data = pd.DataFrame([x.split(',') for x in a_lines], columns=['Type', 'ID', 'ignore', 'Title', 'Stream'])
c_data = pd.DataFrame([x.split(',') for x in c_lines], columns=['Type', 'UserID', 'num'])
ac_lines = []
for line in data:
    if line.startswith('C')|line.startswith('V'):
        ac_lines.append(line)
##将C、V开头的数据整合为以V开头的数据，将用户信息添加到V开头的数据后
result = []
temp = []
for line in ac_lines:
    if line.startswith('C'):
        if temp:
            result.append(temp)
        temp = [line]
    else:
        temp.append(line)
if temp:
    result.append(temp)

fw_data = []
for group in result:
    c_line1 = group[0].split(',')
    for v_line1 in group[1:]:
        fw_data.append(v_line1 + ',' + c_line1[1])
fw_dt = pd.DataFrame([x.split(',') for x in fw_data], columns=['Type', 'UserID', 'ign', 'ID'])

a_attri = ['ID', 'Title', 'Stream']
c_attri = ['UserID']
fw_attri = ['UserID', 'ID']

web_data = a_data[a_attri]
print("网站数据如下：\n",web_data)

网站数据如下：
        ID                              Title           Stream
0    1287          "International AutoRoute"   "/autoroute"\n
1    1288                          "library"     "/library"\n
2    1289  "Master Chef Product Information"  "/masterchef"\n
3    1297                  "Central America"    "/centroam"\n
4    1215         "For Developers Only Info"   "/developer"\n
..    ...                                ...              ...
289  1219    "Corporate Advertising Content"         "/ads"\n
290  1030                "Windows NT Server"    "/ntserver"\n
291  1182                          "Fortran"     "/fortran"\n
292  1100                  "MS in Education"   "/education"\n
293  1210                      "SNA Support"  "/snasupport"\n

[294 rows x 3 columns]


1 数据预处理

In [5]:
user_data = c_data[c_attri]
print("用户数据如下：\n",user_data)

用户数据如下：
         UserID
0      "10001"
1      "10002"
2      "10003"
3      "10004"
4      "10005"
...        ...
32706  "42707"
32707  "42708"
32708  "42709"
32709  "42710"
32710  "42711"

[32711 rows x 1 columns]


In [6]:
visit_data =fw_dt[fw_attri]
print("用户访问网站数据如下：\n",visit_data)

用户访问网站数据如下：
       UserID       ID
0       1000  "10001"
1       1001  "10001"
2       1002  "10001"
3       1001  "10002"
4       1003  "10002"
...      ...      ...
98649   1003  "42709"
98650   1035  "42710"
98651   1001  "42710"
98652   1018  "42710"
98653   1008  "42711"

[98654 rows x 2 columns]


In [7]:
visit_data =fw_dt[fw_attri]
print("用户访问网站数据如下：\n",visit_data)

用户访问网站数据如下：
       UserID       ID
0       1000  "10001"
1       1001  "10001"
2       1002  "10001"
3       1001  "10002"
4       1003  "10002"
...      ...      ...
98649   1003  "42709"
98650   1035  "42710"
98651   1001  "42710"
98652   1018  "42710"
98653   1008  "42711"

[98654 rows x 2 columns]


In [8]:
ID_counts = visit_data["ID"].value_counts()
print('页面访问量分布：\n',ID_counts)

页面访问量分布：
 ID
"40310"    35
"25185"    31
"12147"    30
"10348"    28
"25922"    28
           ..
"31467"     1
"31471"     1
"31472"     1
"31473"     1
"42711"     1
Name: count, Length: 32711, dtype: int64


2 关联规则挖掘

In [9]:
visit_data["ID"] = visit_data["ID"].str.strip('"')
basket = (visit_data.groupby(["UserID", "ID"])["ID"]
          .count().unstack().reset_index().fillna(0)
          .set_index("UserID"))

frequent_itemsets = apriori(basket, min_support=0.05, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

print("频繁项集：")
print(frequent_itemsets)

print("关联规则：")
print(rules)

频繁项集：
      support        itemsets
0    0.059649         (10021)
1    0.080702         (10068)
2    0.056140         (10132)
3    0.059649         (10156)
4    0.063158         (10208)
..        ...             ...
161  0.052632  (33341, 25922)
162  0.056140  (40310, 25922)
163  0.052632  (41066, 25922)
164  0.056140  (40310, 36955)
165  0.052632  (40310, 41066)

[166 rows x 2 columns]
关联规则：
   antecedents consequents  antecedent support  consequent support   support   
0      (12147)     (10348)            0.105263            0.098246  0.056140  \
1      (10348)     (12147)            0.098246            0.105263  0.056140   
2      (10348)     (25922)            0.098246            0.098246  0.056140   
3      (25922)     (10348)            0.098246            0.098246  0.056140   
4      (40310)     (10348)            0.122807            0.098246  0.052632   
5      (10348)     (40310)            0.098246            0.122807  0.052632   
6      (10454)     (40310)            0.0771

In [11]:
print("支持度：")
print(rules["support"])

print("置信度：")
print(rules["confidence"])

print("提升度：")
print(rules["lift"])

支持度：
0     0.056140
1     0.056140
2     0.056140
3     0.056140
4     0.052632
5     0.052632
6     0.052632
7     0.052632
8     0.059649
9     0.059649
10    0.052632
11    0.052632
12    0.052632
13    0.052632
14    0.052632
15    0.052632
16    0.056140
17    0.056140
18    0.052632
19    0.052632
20    0.056140
21    0.056140
22    0.052632
23    0.052632
Name: support, dtype: float64
置信度：
0     0.533333
1     0.571429
2     0.571429
3     0.571429
4     0.428571
5     0.535714
6     0.681818
7     0.428571
8     0.566667
9     0.485714
10    0.535714
11    0.681818
12    0.428571
13    0.483871
14    0.681818
15    0.535714
16    0.457143
17    0.571429
18    0.535714
19    0.535714
20    0.457143
21    0.695652
22    0.428571
23    0.535714
Name: confidence, dtype: float64
提升度：
0     5.428571
1     5.428571
2     5.816327
3     5.816327
4     4.362245
5     4.362245
6     5.551948
7     5.551948
8     4.614286
9     4.614286
10    6.939935
11    6.939935
12    3.940092
13    3